# Tên : Huỳnh Ngọc Công Danh
# MSSV : 19521322

In [182]:
!pip install geopandas
!pip install matplotlib==3.1.3
!pip install osmnet
!pip install folium
!pip install rtree
!pip install pygeos
!pip install geojson
!git clone https://github.com/CityScope/CSL_HCMC

fatal: destination path 'CSL_HCMC' already exists and is not an empty directory.


In [183]:
import geopandas as gpd
import folium
import pandas as pd
import matplotlib.pyplot as plt
import json
df = gpd.read_file('/content/CSL_HCMC/Data/GIS/Population/population_HCMC/population_shapefile/Population_District_Level.shp')
df.head()

,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,geometry
0,District 1,760,District,180225.0,187435.0,187419.0,190943.0,197421.0,200297.0,193632.0,208475.0,211220.0,142625.0,23360.40,24294.90,24292.90,24749.60,25589.30,25962.10,25098.20,27022.1,27377.90,18486.8,15274.835115,7.714899e+06,"POLYGON ((684948.318 1193882.223, 684971.256 1..."
1,District 3,770,District,190553.0,188945.0,190631.0,191580.0,193206.0,193694.0,196333.0,198104.0,199095.0,190375.0,38688.60,38362.10,38704.40,38897.10,39227.20,39326.30,39862.10,40221.7,40422.90,38652.5,12182.058264,4.925215e+06,"POLYGON ((684382.122 1193521.050, 684410.753 1..."
2,District 4,773,District,180980.0,183261.0,184711.0,185649.0,185808.0,187157.0,186727.0,186995.0,183894.0,175329.0,43311.10,43857.00,44204.00,44428.40,44466.50,44789.30,44686.40,44750.6,44008.40,41958.7,9483.716891,4.178668e+06,"POLYGON ((687625.438 1189132.516, 687615.280 1..."
3,District 6,775,District,249329.0,253474.0,254212.0,257865.0,266121.0,257183.0,258945.0,260742.0,262757.0,233561.0,34910.50,35490.80,35594.20,36105.70,37261.60,36010.20,36256.90,36508.5,36790.60,32702.7,13077.365262,7.140945e+06,"POLYGON ((679534.413 1187518.498, 679077.904 1..."
4,District 7,778,District,244276.0,274828.0,268483.0,266633.0,280743.0,296757.0,310178.0,317488.0,327223.0,360155.0,6929.39,7796.06,7616.07,7563.59,7963.85,8418.12,8798.84,9006.2,9282.35,10216.5,33260.915360,3.525220e+07,"POLYGON ((691506.715 1190058.151, 691359.893 1..."


In [184]:
pop=df['Pop_2019'] / df['Pop_2017']
df['pop'] = pop
df.head()

,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,geometry,pop
0,District 1,760,District,180225.0,187435.0,187419.0,190943.0,197421.0,200297.0,193632.0,208475.0,211220.0,142625.0,23360.40,24294.90,24292.90,24749.60,25589.30,25962.10,25098.20,27022.1,27377.90,18486.8,15274.835115,7.714899e+06,"POLYGON ((684948.318 1193882.223, 684971.256 1...",0.675244
1,District 3,770,District,190553.0,188945.0,190631.0,191580.0,193206.0,193694.0,196333.0,198104.0,199095.0,190375.0,38688.60,38362.10,38704.40,38897.10,39227.20,39326.30,39862.10,40221.7,40422.90,38652.5,12182.058264,4.925215e+06,"POLYGON ((684382.122 1193521.050, 684410.753 1...",0.956202
2,District 4,773,District,180980.0,183261.0,184711.0,185649.0,185808.0,187157.0,186727.0,186995.0,183894.0,175329.0,43311.10,43857.00,44204.00,44428.40,44466.50,44789.30,44686.40,44750.6,44008.40,41958.7,9483.716891,4.178668e+06,"POLYGON ((687625.438 1189132.516, 687615.280 1...",0.953424
3,District 6,775,District,249329.0,253474.0,254212.0,257865.0,266121.0,257183.0,258945.0,260742.0,262757.0,233561.0,34910.50,35490.80,35594.20,36105.70,37261.60,36010.20,36256.90,36508.5,36790.60,32702.7,13077.365262,7.140945e+06,"POLYGON ((679534.413 1187518.498, 679077.904 1...",0.888886
4,District 7,778,District,244276.0,274828.0,268483.0,266633.0,280743.0,296757.0,310178.0,317488.0,327223.0,360155.0,6929.39,7796.06,7616.07,7563.59,7963.85,8418.12,8798.84,9006.2,9282.35,10216.5,33260.915360,3.525220e+07,"POLYGON ((691506.715 1190058.151, 691359.893 1...",1.100641


In [185]:
df_top10 = df.nlargest(10,'pop')
df_top10['Dist_Name']

16     District 9
23        Hoc Mon
18         Nha Be
22    District 12
17     District 2
13         Cu Chi
4      District 7
15        Thu Duc
14     Binh Chanh
12       Binh Tan
Name: Dist_Name, dtype: object

In [186]:
df_top10.crs

<Projected CRS: EPSG:32648>
Name: WGS 84 / UTM zone 48N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 102°E and 108°E, northern hemisphere between equator and 84°N, onshore and offshore. Cambodia. China. Indonesia. Laos. Malaysia - West Malaysia. Mongolia. Russian Federation. Singapore. Thailand. Vietnam.
- bounds: (102.0, 0.0, 108.0, 84.0)
Coordinate Operation:
- name: UTM zone 48N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [187]:
click = gpd.read_file('/content/response3.json')
click.head(10)

,dbh,geometry
0,1,POINT (106.71136 10.85113)
1,1,POINT (106.71695 10.74449)
2,1,POINT (106.66315 10.83289)
3,1,POINT (106.66267 10.83339)
4,1,POINT (106.65977 10.83729)
5,1,POINT (106.75131 10.85530)
6,1,POINT (106.66677 10.79108)
7,1,POINT (106.68205 10.82357)
8,1,POINT (106.73263 10.74918)
9,1,POINT (106.73554 10.72743)


In [188]:
#convert sang he truc toa do x,y 
click_crs = click.to_crs(epsg=32648)
click_crs.head(10)


,dbh,geometry
0,1,POINT (687074.226 1200044.747)
1,1,POINT (687752.360 1188252.815)
2,1,POINT (681813.990 1197998.313)
3,1,POINT (681761.573 1198052.679)
4,1,POINT (681442.508 1198483.069)
5,1,POINT (691440.136 1200531.193)
6,1,POINT (682235.125 1193375.153)
7,1,POINT (683886.534 1196978.420)
8,1,POINT (689464.643 1188780.763)
9,1,POINT (689796.719 1186376.959)


In [189]:
data = gpd.sjoin(click_crs,df_top10 , how="left", op="within")
data = data.dropna()
data.head(1)

,dbh,geometry,index_right,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,pop
0,1,POINT (687074.226 1200044.747),15.0,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,507650.0,517772.0,528413.0,537050.0,542612.0,592686.0,9259.21,9546.55,10028.2,10487.8,10630.2,10842.2,11065.0,11245.9,11362.3,12410.9,52828.50143,4.775581e+07,1.092283


In [190]:
import numpy as np
points = data[['geometry']].values
points = [np.array(p[0]) for p in points]
points_np = np.array(points)

In [191]:
from sklearn.cluster import KMeans
import numpy as np
k = KMeans(n_clusters=20, random_state=0).fit(points_np)
k.labels_

array([ 3,  8,  1, ...,  8,  1, 17], dtype=int32)

In [192]:
data["labels"] = k.labels_
data.head(1)

,dbh,geometry,index_right,Dist_Name,Dist_ID,Level,Pop_2009,Pop_2010,Pop_2011,Pop_2012,Pop_2013,Pop_2014,Pop_2015,Pop_2016,Pop_2017,Pop_2019,Den_2009,Den_2010,Den_2011,Den_2012,Den_2013,Den_2014,Den_2015,Den_2016,Den_2017,Den_2019,Shape_Leng,Shape_Area,pop,labels
0,1,POINT (687074.226 1200044.747),15.0,Thu Duc,762,District,442177.0,455899.0,478900.0,500850.0,507650.0,517772.0,528413.0,537050.0,542612.0,592686.0,9259.21,9546.55,10028.2,10487.8,10630.2,10842.2,11065.0,11245.9,11362.3,12410.9,52828.50143,4.775581e+07,1.092283,3


In [193]:
df_clean = data[['Dist_Name','geometry','labels']]
df_clean.head(1)

,Dist_Name,geometry,labels
0,Thu Duc,POINT (687074.226 1200044.747),3


In [194]:
df_clean = df_clean.to_crs(epsg=4326)
df_clean.head(1)

,Dist_Name,geometry,labels
0,Thu Duc,POINT (106.71136 10.85113),3


In [195]:
d = df_clean.groupby('Dist_Name')['labels'].value_counts()
label_max = []
dist = []
tmp = ""
for i in d.keys():
  if tmp != i[0]:
    tmp = i[0]
    label_max.append(i[1])
    dist.append(i[0])
  else:
    continue
print(label_max)
print(dist)


[14, 9, 4, 3, 12, 0, 13, 16, 2, 10]
['Binh Chanh', 'Binh Tan', 'Cu Chi', 'District 12', 'District 2', 'District 7', 'District 9', 'Hoc Mon', 'Nha Be', 'Thu Duc']


In [196]:
point = []
for i in range(0,len(dist)):
  t = df_clean[(df_clean["Dist_Name"] == dist[i]) & (df_clean["labels"] == label_max[i])]
  for i in t.geometry.values:
    a = [i.y,i.x]
    point.append(a)
point = np.array(point)
point

array([[ 10.66951754, 106.60711102],
       [ 10.67535521, 106.64462842],
       [ 10.6624745 , 106.64336141],
       ...,
       [ 10.84788121, 106.75122356],
       [ 10.84811677, 106.75629356],
       [ 10.84412247, 106.75441658]])

In [197]:
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
m = folium.Map(location=[10.8, 106.7], zoom_start=10, tiles='CartoDB positron')
m

In [198]:
from folium.plugins import HeatMap
heatmap_layer = HeatMap(point, 
                radius=8, max_zoom=10)
heatmap_layer.add_to(m)
m

In [207]:
# Save image
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
import os
import imgkit

import time
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
mapFname = 'output.html'
m.save(mapFname)
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=mapFname)
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=options)
# Path sampleFile = Paths.get("o.html");
driver.get(tmpurl);
time.sleep(5)
driver.save_screenshot('output.png')
driver.quit()
# print(tmpurl)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:15 http://archive.ubuntu.com/ubuntu bionic-